In [1]:
from datasets import LogisticRegressionDataset
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_FEATURES = 4
data = LogisticRegressionDataset(num_points=1000, num_features=NUM_FEATURES)

In [3]:
data.true_weights # TODO: try this experiment with the same weights for each feature, also maybe the gaussian. Also try sampling from the logit probability instead of just picking >0.5

array([0.09762701, 0.43037873, 0.20552675, 0.08976637])

In [4]:
1 / (1 + np.exp(-data.train_data.dataset.tensors[0][:10] @ data.true_weights))

tensor([0.6850, 0.6403, 0.6561, 0.8661, 0.8730, 0.4739, 0.3581, 0.3471, 0.8440,
        0.4021], dtype=torch.float64)

In [5]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], 1 / (1 + np.exp(-data.train_data.dataset.tensors[0][:10] @ data.true_weights)), data.train_data.dataset.tensors[1][:10]

(tensor([[-0.7635,  1.4589, -0.6241,  3.9177],
         [ 4.6366, -1.1656,  2.9173,  0.2889],
         [ 0.6804,  4.2560, -4.2896, -4.1287],
         [-4.7978,  3.3262,  2.7816,  3.7001],
         [ 4.7862,  2.9916, -0.3852,  2.8053],
         [-3.8173,  1.3992, -3.5665,  4.4467],
         [ 0.2185, -0.8534, -2.3544,  2.7423],
         [-0.4385,  0.6843, -4.8121,  1.1764],
         [ 1.1210,  1.1693,  4.4375,  1.8182],
         [-1.4049, -0.6297,  1.9763, -4.3977]], dtype=torch.float64),
 tensor([0.6850, 0.6403, 0.6561, 0.8661, 0.8730, 0.4739, 0.3581, 0.3471, 0.8440,
         0.4021], dtype=torch.float64),
 tensor([1, 1, 1, 1, 1, 0, 0, 0, 1, 0]))

In [6]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 1    353
0    347
dtype: int64
Val data:
 0    51
1    49
dtype: int64
Test data:
 0    108
1     92
dtype: int64


In [7]:
train_X, train_y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=len(data.train_data))))
test_X, test_y = next(iter(torch.utils.data.DataLoader(data.test_data, batch_size=len(data.test_data))))
train_X, train_y

(tensor([[-3.3531,  1.2148,  0.7723, -2.6211],
         [-3.3751,  1.1556, -3.7618,  3.4801],
         [ 4.8280, -0.0054, -3.6371,  1.8629],
         ...,
         [-1.5137, -3.9215, -1.1141, -0.5232],
         [ 2.1686, -1.0394,  0.6542, -3.1672],
         [ 0.5219,  0.8448,  4.6194, -2.0785]], dtype=torch.float64),
 tensor([1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
         1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
         0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
         1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
         0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
         1, 1, 1, 1, 0, 0, 1, 0, 

In [8]:
from sklearn.linear_model import LogisticRegression
m = LogisticRegression(fit_intercept=False)
m.fit(X=train_X, y=train_y)
m.score(X=test_X, y=test_y)

1.0

In [9]:
m.coef_

array([[1.08271806, 4.65796367, 2.17886376, 0.93023272]])

In [10]:
test_X[:10], test_y[:10]

(tensor([[ 2.2525,  0.0132,  4.5608,  1.4399],
         [-0.9463, -1.7896, -4.7005,  2.3725],
         [-1.0477, -1.8916,  2.1863, -1.6402],
         [ 2.3193, -1.5777, -1.2441, -1.4089],
         [ 1.9948, -2.0256,  3.1380, -1.0349],
         [ 3.2347,  3.6613,  4.6081, -4.3488],
         [-2.7559,  4.5368,  0.8232, -3.9253],
         [ 3.0076, -1.2257,  1.7024, -0.2834],
         [ 0.7115,  4.1469,  2.3864,  3.5667],
         [ 3.5886,  1.0539,  0.6609,  0.7584]], dtype=torch.float64),
 tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1]))

In [11]:
np.array_equal(
    (1 / (1 + np.exp(-test_X[:10] @ m.coef_.T)) > 0.5).view(-1), 
    test_y[:10]
)

True

In [12]:
# from brunoflow.net import LogReg
# m_bf = LogReg(input_size=NUM_FEATURES, bias=False)
# m_bf.